In [1]:
import transformers
import pandas as pd
import numpy as np
import nltk
from utils import TweetsDataset
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained("Newtral/xlm-r-finetuned-toxic-political-tweets-es")
model = transformers.XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=4, problem_type="multi_label_classification")
df = pd.read_csv("../../data/procesed.csv")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [9]:
df_train, df_valid = TweetsDataset.split_test_val(df,test_size=0.20)
train_data_loader = TweetsDataset.create_data_loader(df_train, tokenizer, batch_size=8)
valid_data_loader = TweetsDataset.create_data_loader(df_valid, tokenizer, batch_size=8)

In [10]:
model = model.to(device)
EPOCHS = 3
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_data_loader) * EPOCHS

scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0, 
    num_training_steps = total_steps
)

In [11]:
from collections import defaultdict
from utils import Model

history = defaultdict(list)

Model.train(EPOCHS, model, train_data_loader, optimizer, device, scheduler, history, len(df_train), len(df_valid), valid_data_loader)

Epoch 1/3
----------
Train loss: 0.54 Accuracy: 0.345
Validation loss 0.535 Accuracy: 0.364 F1 score: 0.364

Epoch 2/3
----------
Train loss: 0.537 Accuracy: 0.352
Validation loss 0.535 Accuracy: 0.364 F1 score: 0.364

Epoch 3/3
----------
Train loss: 0.536 Accuracy: 0.358
Validation loss 0.535 Accuracy: 0.364 F1 score: 0.364



defaultdict(list,
            {'train_acc': [0.34509803921568627,
              0.35171568627450983,
              0.3575980392156863],
             'train_loss': [0.5398042104407853,
              0.5369225340341431,
              0.5360109250140346],
             'val_acc': [0.3638888888888889,
              0.3638888888888889,
              0.3638888888888889],
             'val_loss': [0.5351561899538393,
              0.5350650320450465,
              0.5347867805648733],
             'f_score': [0.3638888888888888,
              0.3638888888888888,
              0.3638888888888888]})

In [12]:
torch.save(model.state_dict(), 'xlm_newtral_pretrained.bin')

In [4]:
# Test
from utils import Results, Model
model = transformers.XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=4, problem_type="multi_label_classification")
model.to(device)
model.load_state_dict(torch.load('xlm_newtral_pretrained.bin'))
df_test = pd.read_csv('../../data/development_test.csv')
df_test = pd.DataFrame({
    "tweets": df_test["tweet"],
    "labels": df_test["ideology_multiclass"].map({'moderate_left': 0, 'moderate_right': 1, "left": 2, "right": 3})
})

test_data_loader = TweetsDataset.create_data_loader(df_test, tokenizer)
acc, loss, f1 = Model.test(model, test_data_loader, device, len(df_test))
Results.add_result("newtral-xml", "ideology-multiclass", acc, loss, f1)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

Test loss: 0.532 Accuracy: 0.4
